In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

In [24]:
dataset = pd.read_csv('train.csv')
dftrain, dfeval = train_test_split(dataset, test_size=0.1, random_state=42)
dftrain = pd.read_csv('train.csv')

y_train = dftrain.pop('churn')
y_eval = dfeval.pop('churn')


In [3]:

import tensorflow as tf
tf.random.set_seed(123)

fc = tf.feature_column

In [4]:
CATEGORICAL_COLUMNS = list(dftrain.select_dtypes(include='object'))
NUMERIC_COLUMNS = list(dftrain.select_dtypes(include=['float64','int64']))


dftrain[NUMERIC_COLUMNS] = dftrain[NUMERIC_COLUMNS].fillna(dftrain[NUMERIC_COLUMNS].mean())
dftrain[CATEGORICAL_COLUMNS] = dftrain[CATEGORICAL_COLUMNS].fillna("Y")
dfeval[NUMERIC_COLUMNS] = dfeval[NUMERIC_COLUMNS].fillna(dfeval[NUMERIC_COLUMNS].mean())
dfeval[CATEGORICAL_COLUMNS] = dfeval[CATEGORICAL_COLUMNS].fillna("Y")


C:\Users\Sergey\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [5]:
def one_hot_cat_column(feature_name, vocab):
  return tf.feature_column.indicator_column(
      tf.feature_column.categorical_column_with_vocabulary_list(feature_name,
                                                 vocab))
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  # Need to one-hot encode categorical features.
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(one_hot_cat_column(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                           dtype=tf.float32))

In [6]:
NUM_EXAMPLES = len(y_train)

def make_input_fn(X, y, n_epochs=None, shuffle=True):
  def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
    if shuffle:
      dataset = dataset.shuffle(NUM_EXAMPLES)
    # For training, cycle thru dataset as many times as need (n_epochs=None).
    dataset = dataset.repeat(n_epochs)
    # In memory training doesn't use batching.
    dataset = dataset.batch(NUM_EXAMPLES)
    return dataset
  return input_fn

In [7]:
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, shuffle=False, n_epochs=1)

In [8]:
linear_est = tf.estimator.LinearClassifier(feature_columns,
    optimizer=lambda: tf.keras.optimizers.Ftrl(
        learning_rate=tf.exponential_decay(
            learning_rate=0.1,
            global_step=tf.get_global_step(),
            decay_steps=10000,
            decay_rate=0.96))
clear_output()
print("fssfsdf")
# Train model.
linear_est.train(train_input_fn, max_steps=10)
print("fssfsdf")

SyntaxError: invalid syntax (<ipython-input-8-c2f68d158d33>, line 8)

In [ ]:


# Since data fits into memory, use entire dataset per layer. It will be faster.
# Above one batch is defined as the entire dataset.
n_batches = 1
est = tf.estimator.BoostedTreesClassifier(feature_columns,
                                          n_batches_per_layer=n_batches)


In [ ]:
est.train(train_input_fn, max_steps=10)

# Eval.
result = est.evaluate(eval_input_fn)
clear_output()
print(pd.Series(result))

pred_dicts = list(est.predict(eval_input_fn))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')
plt.show()

from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_eval, probs)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.xlim(0,)
plt.ylim(0,)
plt.show()

In [25]:
from scipy.spatial.distance import euclidean
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(3,3), analyzer='char_wb')

n1, n2, n3, n4 = vect.fit_transform(['andersen', 'petersen', 'petrov', 'smith']).toarray()


In [26]:
CATEGORICAL_COLUMNS = list(dataset.select_dtypes(include='object'))

In [27]:

dataset = pd.get_dummies(dataset, columns=CATEGORICAL_COLUMNS)

In [28]:
dataset.head()

,Customer_ID,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,...,kid3_5_U,kid3_5_Y,kid6_10_U,kid6_10_Y,kid11_15_U,kid11_15_Y,kid16_17_U,kid16_17_Y,creditcd_N,creditcd_Y
0,94739,44.7725,441.0,39.5475,0.4950,24.75,9.9000,9.9000,0.0,0.00,...,1,0,1,0,1,0,0,1,0,1
1,3355,37.8950,178.5,30.0000,0.0000,15.00,5.2500,5.2500,0.0,0.00,...,1,0,1,0,0,1,0,1,0,1
2,400,96.3350,1852.5,75.0000,0.7425,44.25,4.4250,4.4250,0.0,1.74,...,1,0,1,0,1,0,1,0,0,1
3,79745,42.6750,410.0,44.9900,0.2475,21.25,7.4375,7.4375,0.0,0.00,...,1,0,1,0,1,0,1,0,0,1
4,37406,30.7000,119.0,29.9900,0.0000,0.00,0.0000,0.0000,0.0,0.71,...,1,0,1,0,1,0,1,0,0,1


In [29]:
for column in list(dataset):
    dataset[column] = dataset[column].fillna(dataset[column].median())
    



In [30]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Columns: 231 entries, Customer_ID to creditcd_Y
dtypes: float64(69), int64(10), uint8(152)
memory usage: 56.1 MB


In [32]:
dropped_columns = []

corr_matrix = dataset.corr(method='spearman')

for i in corr_matrix:
    if abs(corr_matrix.drop([i], axis=0)[i].max()) > 0.8:
        dropped_columns.append(i)
        corr_matrix = corr_matrix.drop([i], axis=1)
        corr_matrix = corr_matrix.drop([i], axis=0)
dataset = dataset.drop(dropped_columns, axis=1)

KeyboardInterrupt: 

In [ ]:
dataset.info()

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(dataset, test_size=0.01, random_state=42)
train_X = train.drop(['churn'], axis=1)
train_y = train.churn

In [ ]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
import pandas as pd

In [ ]:
model = RandomForestClassifier(n_estimators=450, criterion='entropy',
                                                     max_depth=450, min_samples_leaf=11,
                                                     min_samples_split=20, random_state=42)
model.fit(train_X, train_y)

In [ ]:
pred_val = model.predict(val.drop(['churn'], axis=1))
print("Roc val Random Forest: " + str(roc_auc_score(val.churn, pred_val)))

In [ ]:
print(pd.DataFrame(
            {'features': train_X.columns, 'feature_important': model.feature_importances_}).sort_values(
            'feature_important'))

In [1]:
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
data_train = pd.read_csv('train.csv')

ImportError: cannot import name 'CatBoostClassifier' from 'catboost' (C:\Users\Sergey\PycharmProjects\ottok\catboost.py)